# Obtaining the Dataset


(no ML in entire section)

### Spotify API

In [ ]:
import json
import requests
import base64

client id and secret are unique to registered Spotify web app, code is temporary and retrieved from separate Flask app

In [ ]:
client_id = ___________
client_secret = ____________
code = 'AQAIxrVr21IodwATJvNxWh7E69FXAg-XG0uVnmtQL5Q5-MViWtDd5Bd_QHAYa2M2u9YXzMJw0P4jIwrGOnS2e1Rb_dYqC7DvAZbbyF6HWm0KZyC76FLJZhoLh-76sq_XGQMpQYgjAcorIRy-p4AINNNRPOEcw-szJsZDqMDnsTIhen0sOPYKiCNPGCymBF8Q4iFh9vyU0hQ8JKGd5S1VXbmjasJgKAxKjmxOOYLoYuUba8dAVuUk4J0mqyKVUw'

In [ ]:
# # NOTE: THESE ARE FOR USER DATA, NOT GENERAL SEARCH
# auth_header = base64.urlsafe_b64encode(f'{client_id}:{client_secret}'.encode('ascii'))
# token_url = "https://accounts.spotify.com/api/token"
# token_data = {
#     'grant_type':'authorization_code',
#     'code': code,
#     'redirect_uri': 'http://127.0.0.1:5000/' # running the get_code.py program locally
# }
# token_headers = {
#     # "Authorization": f"Basic {encoded_client_creds}"
#     'Authorization': f'Basic {auth_header.decode("ascii")}' # or just encoded_client_creds here
# }
# r = requests.post(token_url, data=token_data, headers=token_headers)
# print(r.json()) # not needed until retrieving from user data
# token_response_data = r.json()

Retrieve songs and lyrics

In [ ]:
auth_header = base64.urlsafe_b64encode(f'{client_id}:{client_secret}'.encode('ascii'))
token_url = "https://accounts.spotify.com/api/token"
token_data = {
    'grant_type':'client_credentials',
    'code': code,
    'redirect_uri': 'http://127.0.0.1:5000/' # running the get_code.py program locally
}
token_headers = {
    # "Authorization": f"Basic {encoded_client_creds}"
    'Authorization': f'Basic {auth_header.decode("ascii")}' # or just encoded_client_creds here
}

In [ ]:
r = requests.post(token_url, data=token_data, headers=token_headers)
print(r.json()) # note: diff from user api functions; only searches publically available data
token_response_data = r.json()

{'access_token': 'BQD3akUf_QBxZ3UsvhoIBu4ZD_Ylgmvp0oqwgf0d9Wby0hqR-DBZ2MkWd1kzgNHfcFnfq5eZMSHsxTxcjC3xZ6GFhtoRz7v0pdUcbA27WwOl_M_MZFw', 'token_type': 'Bearer', 'expires_in': 3600}


In [ ]:
print(r.json()['access_token'])

BQD3akUf_QBxZ3UsvhoIBu4ZD_Ylgmvp0oqwgf0d9Wby0hqR-DBZ2MkWd1kzgNHfcFnfq5eZMSHsxTxcjC3xZ6GFhtoRz7v0pdUcbA27WwOl_M_MZFw


In [ ]:
# Green Day, The Strokes, The Velvet Underground, Slaughter Beach, Dog, Fleetwood Mac, Mitski, and boygenius
# 0          1            2                       3                     4              5           6

# got these manually from artist's IDs in their share links
artist_ids = ['7oPftvlwr6VrsViSDV7fJY', '0epOFNiUfyON9EYx7Tpr6V', '1nJvji2KIlWSseXRSlNYsC', '3lWVgSwutPsiJ8Awm7OTKU', '08GQAI4eElDnROBrJRGE0X', '2uYWxilOVlUdk4oV9DvwqK', '1hLiboQ98IQWhpKeP9vRFw']
headers = {
    'Authorization': f"Bearer {token_response_data['access_token']}",
    'Content-Type': 'application/json'
}

In [ ]:
# loop through artists to retrieve albums
responses=[]
for id in artist_ids:
  responses.append(requests.get(f"https://api.spotify.com/v1/artists/{id}/albums", headers=headers).json())

In [ ]:
# responses # full api response

In [ ]:
responses[0]['items'][0]['id']

'47l8TkJCLzHaekHImz1e9g'

Each entry in `responses` is a response containing all of artist `i`'s albums. Loop through `responses` and replace the response by a list of album Spotify IDs

In [ ]:
album_ids = []
for i in range(len(responses)):
  album_list = []
  print("len(responses[i]['items']):", len(responses[i]['items']))
  for j in range(len(responses[i]['items'])):
    album_list.append(responses[i]['items'][j]['id'])
  album_ids.append(album_list)

len(responses[i]['items']): 20
len(responses[i]['items']): 20
len(responses[i]['items']): 20
len(responses[i]['items']): 20
len(responses[i]['items']): 20
len(responses[i]['items']): 16
len(responses[i]['items']): 20


In [ ]:
# album_ids # list of list of green day album ids

Retrieve all track IDs for each album

In [ ]:
track_ids = []
for artist_discog in album_ids: # there are 7 artist_discogs, each being a list
  tracks = []
  for album_id in artist_discog:
    album_tracks = requests.get(f"https://api.spotify.com/v1/albums/{album_id}/tracks", headers=headers).json()
    tracks.append(album_tracks)
  track_ids.append(tracks)

In [ ]:
# track_ids

In [ ]:
# 3rd artist tracks, 1st album, 1st song id
track_ids[2][0]['items'][0]['id']

'3SFayZBTps4PWfSIc8Pil5'

Compile all track ID's into a single list for each artist

In [ ]:
bare_track_ids = []
for i in range(len(track_ids)): # loop through artists
  ids = []
  for j in range(len(track_ids[i])): # loop through albums
    for k in range(len(track_ids[i][j]['items'])):
      ids.append(track_ids[i][j]['items'][k]['id'])
  bare_track_ids.append(ids)

In [ ]:
# bare_track_ids[0] # list of green day song ids

In [ ]:
# print num songs for each artist
for artist in bare_track_ids:
  for track_id in artist:
    song = requests.get(f"https://api.spotify.com/v1/tracks/{track_id}", headers=headers).json()
    if song['album']['artists'][0]['id'] not in artist_ids:
      artist.remove(track_id)

### Lyric API

Isolate words from API response

In [ ]:
lyricurl = 'https://spotify-lyric-api-984e7b4face0.herokuapp.com/?trackid=6PmbpHTKSmKP3FuMIbjttI' #sub
lyric_data = {
    "trackid": '6PmbpHTKSmKP3FuMIbjttI' #sub
}
lyric_response = requests.get(lyricurl, data=lyric_data)
lyric_response.json()

{'error': False,
 'syncType': 'LINE_SYNCED',
 'lines': [{'startTimeMs': '12150',
   'words': "Where's the harm in voicing a doubt?",
   'syllables': [],
   'endTimeMs': '0'},
  {'startTimeMs': '17160',
   'words': "You'll find me in the lavatory",
   'syllables': [],
   'endTimeMs': '0'},
  {'startTimeMs': '21120',
   'words': "And where's the harm in talking out loud",
   'syllables': [],
   'endTimeMs': '0'},
  {'startTimeMs': '26190',
   'words': "When I'm on my own?",
   'syllables': [],
   'endTimeMs': '0'},
  {'startTimeMs': '31130',
   'words': "What's so wrong with reading my stars",
   'syllables': [],
   'endTimeMs': '0'},
  {'startTimeMs': '36150',
   'words': "When I'll be in the lavatory?",
   'syllables': [],
   'endTimeMs': '0'},
  {'startTimeMs': '40140',
   'words': 'And what is so wrong with counting the cars',
   'syllables': [],
   'endTimeMs': '0'},
  {'startTimeMs': '46280',
   'words': "When I'm all alone?",
   'syllables': [],
   'endTimeMs': '0'},
  {'startTime

In [ ]:
words = ""
for line in lyric_response.json()['lines']:
  if (line['words'] != '♪') or (line['words'] != ''):
    words+=line['words'] + " / "

In [ ]:
words

"Where's the harm in voicing a doubt? / You'll find me in the lavatory / And where's the harm in talking out loud / When I'm on my own? / What's so wrong with reading my stars / When I'll be in the lavatory? / And what is so wrong with counting the cars / When I'm all alone? / You're not the only one that I know / And I'm too proud to talk to you anyway / You're not the only one that I know / And I'm far too proud to talk to you any day / So, I say I'm in love with the world / And what is so wrong with voicing a doubt / When I'm on my own? / It's perfectly fine to sleep in a chair / From Monday 'til Saturday / And what is so wrong with talking out loud / When I'm all alone? / You're not the only one that I know / And I'm too proud to talk to you anyway / You're, you're not the only one that I know / But I'm far too proud to talk to you any day / So they rode out west to the seaside / And they gladly decided to stay / After two hours wandering outside / Ooh, the sea air drove them away,

# Input Data

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(columns=['label','lyrics'])
df.head()

,label,lyrics


In [ ]:
for i in range(len(bare_track_ids)):
  for id in bare_track_ids[i]:
    lyric_response = requests.get(f"https://spotify-lyric-api-984e7b4face0.herokuapp.com/?trackid={id}", data={"trackid": f"{id}"})
    words = ""
    if not lyric_response.json()['error']:
      for line in lyric_response.json()['lines']:
        if (line['words'] != '♪') or (line['words'] != ''):
          words+=line['words'] + " / "
    df.loc[len(df.index)] = [i, words]

In [ ]:
len(df.index)

1441

In [ ]:
df['label'].value_counts()

4    335
0    289
2    254
6    183
3    133
1    130
5    117
Name: label, dtype: int64

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df.to_csv('/content/drive/My Drive/mydata.csv', index=False)

Mounted at /content/drive


In [ ]:
df[df['label']==1].head(5) # the strokes

,label,lyrics
305,1,Up on a hill is where we begin / This little s...
306,1,"Last night, she said / ""Oh, baby, I feel so do..."
307,1,Was an honest man / ♪ / Asked me for the phone...
308,1,"Oh! / Ha-ha, I meant ah! / No, I didn't mean t..."
309,1,"Last night, she said / ""Oh, baby, I feel so do..."


In [ ]:
df.loc[307]['lyrics']

'Was an honest man / ♪ / Asked me for the phone, tried to take control / Oh, I don\'t see it that way / I don\'t see it that way / ♪ / We shared some ideas / ♪ / All obsessed with fame, says we\'re all the same / Oh, I don\'t see it that way / I don\'t see it that way / ♪ / Raised in Carolina / ♪ / I\'m not like that / Trying to remind her / When we go back / I missed the last bus, I\'ll take the next train / I\'ll try, but you see, it\'s hard to explain / I say the right thing but act the wrong way / I like it right here, but I cannot stay / I\'m watching TV, forget what I\'m told / Well, I am too young and they are too old / The joke is on you, this place is a zoo / You\'re right, it\'s true / ♪ / He says he can\'t decide / ♪ / I shake my head to say, "everything\'s just great" / Oh, I just can\'t remember / I just can\'t remember / ♪ / Raised in Carolina / She says, "I\'m not like that" / Trying to remind her / When we go back / I say the right thing but act the wrong way / I like i